# Bing Search 기반의 RAG를 활용한 데모

이 데모에서는 GPT3 모델을 사용하여 자연 쿼리를 분석하고 지식 기반을 사용하여 더 많은 정보를 검색하고 질문에 답변하는 방법을 보여줍니다.  
1. Bing Search API 활용: 검색한 결과를 참고하는 방법을 소개합니다. 여기서는 Bing Search API를 활용합니다.

In [88]:
import os
import json
import requests
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name = os.getenv('DEPLOYMENT_NAME')

## 1. Bing Search API를 활용하는 방법

### Start with a natural question  
다음과 같이 LLM이 가지고 있지 않을 수 있는 데이터에 대한 접근이 필요할 때, 검색엔진을 활용할 수 있습니다.

### Step 1: GPT3: What do I need to to answer this question?

In [111]:
# 검색을 위해 필요한 정보를 추출하는 함수 정의
def get_search_info(input_query):
    system_prompt='''
    Answer the following questions as best you can. You have access to the following tools: You can only Korean language.
    Web Search: Use the web to find information
    Question: Natural language-based questions that users want to ask through web searches
    Answer: Describes how to effectively answer people's natural language-based questions and creates optimized search keywords needed for use in the Bing Search API. And create optimized search keywords needed for use in the Bing Search API. The answer format must strictly follow the JSON format below.
    {"Keywords": "Specify three sets of keywords information to search for key data that matches the user's question intent."}
    '''
    user_prompt = f'''
    Question: {input_query}
    '''

    # Azure OpenAI GPT를 프롬프트 엔지니어링의 Few-shot 예시
    prompt = [{"role":"system", "content":system_prompt},
                {"role":"user", "content":"Question: 2024년 한국 총선 날짜가 언제인지 회사를 안 가도 되는지 알려줘."},
                {"role":"assistant", "content":"""{"Keywords": "2024년 한국 총선 날짜 + 2024년 총선 공휴일 여부 + 총선 투표일 휴무 정책"]}"""},
                {"role":"user","content": user_prompt}
            ]
        
    response = client.chat.completions.create(
        model=deployment_name,
        messages = prompt,
        max_tokens=600,
        response_format={"type": "json_object"}
    )

    keywords = json.loads(response.choices[0].message.content)['Keywords']

    return json.loads(response.choices[0].message.content)['Keywords']

### Step 2: Search web for more details  

Bing Search API와 관련된 정보는 다음 Page에서 확인할 수 있습니다.  
https://www.microsoft.com/en-us/bing/apis/bing-web-search-api

In [112]:
# Bing search API를 이용하여 검색 결과를 가져오는 함수 정의
def get_search_result(keywords):
    subscription_key = os.getenv("BING_SEARCH_KEY")
    endpoint = os.getenv("BING_SEARCH_ENDPOINT") + "/v7.0/search"

    query = keywords

    # Construct a request
    mkt = 'ko_KR'
    params = { 'q': query, 'mkt': mkt }
    headers = { 'Ocp-Apim-Subscription-Key': subscription_key }

    # Call the API
    response = requests.get(endpoint, headers=headers, params=params)

    context = ''
    for result in response.json()['webPages']['value']:
        context += result['snippet'] + '\n'

    return context

In [113]:
# 검색 결과를 기반으로 사용자의 질문에 대한 답을 생성하는 함수 정의
def generate_answer(input_query, context):
    system_prompt = """You are an agent who answers the user's questions based on the data provided. You can only Korean language."""
    user_prompt = f"""Question: {input_query}
    Web Search Results: {context}
    Answer: 
    """

    response = client.chat.completions.create(
        model=deployment_name,
        messages = [
            {"role":"system", "content":system_prompt},
            {"role":"user","content": user_prompt}
        ],
        max_tokens=600
    )

    return response.choices[0].message.content

In [114]:
# 사용자로부터 검색 질의어를 입력받아서 결과를 출력하는 함수 정의
def get_input_query(input_query):
    # 검색 질의어 출력
    print("-"*80)
    print("검색질의어: " + input_query)
    print("-"*80)

    # 검색 키워드를 추출하는 함수 호출
    keywords = get_search_info(input_query)
    print("검색키워드: " + keywords)
    print("-"*80)

    # 검색 결과를 가져오는 함수 호출
    context = get_search_result(keywords)
    print("검색결과: " + context)
    print("-"*80)

    # 사용자에게 제공할 답변을 생성하는 함수 호출
    answer = generate_answer(input_query, context)
    print("최종답변: " + answer)
    print("-"*80)

In [115]:
input_query = "2024년 '부처님 오신 날'이 언제야? 쉬는 날 맞아? 공휴일이면 대체 휴무는 언제야?"
get_input_query(input_query)

--------------------------------------------------------------------------------
검색질의어: 2024년 '부처님 오신 날'이 언제야? 쉬는 날 맞아? 공휴일이면 대체 휴무는 언제야?
--------------------------------------------------------------------------------
검색키워드: 2024년 한국 부처님 오신 날 날짜 + 2024년 부처님 오신 날 휴일 여부 + 2024년 대체 공휴일
--------------------------------------------------------------------------------
검색결과: 날짜 요일 공휴일; 2024: 5월 15일: 수요일: 부처님 오신 날: 2025: 5월 8일: 목요일: 부처님 오신 날: 2026: 5월 8일: 금요일: 부처님 오신 날: 이전 연도의 날짜를 보려면 페이지 하단으로 스크롤하십시오.
설날, 부처님오신날, 추석은 한국에서 기념하는 날짜와 중국에서 기념하는 날짜가 달라질 수 있는 공휴일이다. 2023년이 대표적인 예로 홍콩에서는 전날인 양력 5월 26일이 공휴일이었다. 2017년까지는 공식적으로 석가탄신일 [5] 이라고 불렸다. 이 때까지 공식 명칭은 석가탄신일이었지만, 사찰에서는 '부처님오신날'을 사용했다. 4월 하순~5월 상순이나 하순에 오는 경우는 십중팔구 윤달 때문이라고 보면 된다. 4월 말이나 5월 초에 오는 경우는 4월 이후에 윤달이 끼고, 5월 말에 오는 경우는 설날이 늦은 경우가 아닌 이상 3월 이전에 윤달이 낀다고 보면 된다.
부처님 오신 날: 다른이름 (사월) 초파일 ((四月) 初八日) 석가탄신일(釋迦誕辰日) 장소: 동아시아: 형태: 공휴일(대한민국, 중화민국, 마카오, 홍콩) 날짜: 음력 4월 8일 (대한민국) 2023년: 5월 27일 (토) 2024년: 5월 15일 (수) 2025년: 5월 5일 (월) 2026년: 5월 24일 (일) 빈도 .

In [118]:
input_query = "2024년 추석 연휴는 언제야? 연휴 기간은으로 얼만큼 쉴 수 있을지 알려줘."
get_input_query(input_query)

--------------------------------------------------------------------------------
검색질의어: 2024년 추석 연휴는 언제야? 연휴 기간은으로 얼만큼 쉴 수 있을지 알려줘.
--------------------------------------------------------------------------------
검색키워드: 2024년 추석 연휴 날짜 + 2024년 추석 연휴 기간 + 2024년 연휴 휴가 일수 
--------------------------------------------------------------------------------
검색결과: 2024년 대한민국 공휴일. 우리는 현재 2024년 공휴일을 목록으로 가지고 있습니다. 공휴일을 확인하시려면 스크롤을 내려주세요. 인기있는 휴일: 부처님 오신 날. 추석. 현충일.
2024년 대한민국 공휴일 연차사용 꿀팁. 신정은 월요일이기 때문에 12월 30일부터 1월 1일까지 총 3일 동안 연휴 기간을 즐길 수 있으며, 2월에는 설날과 설날 다음 덕분에 총 4일 동안 연휴를 즐길 수 있고, 3월 1일 금요일은 삼일절이기 때문에 3월 1일부터 3월 3 ...
추석 2024, 2025 과 2026. 추석은 한국에서 널리 기념하는 가장 큰 명절 중 하나입니다. 이 공휴일은 가족들이 모여서 대화를 하고 큰 잔치를 하면서 행복과 가족간의 유대감을 쌓는 날입니다. 한국 추석에 추석이 있는 주요 이유는 조상들과 고인을 기리기 위한 것이지만, 공휴일이어서 직장이나 다른 일로 떨어져 사는 가족들이 함께 모이는 목적의 공휴일이기도 합니다. 전통적으로, 추석에 한국인들이 열심히 일한 후에 수확을 걷는 가을 추수를 축하하기도 합니다. 추석은 음력 8월 15일 또는 추분점에 기념합니다. 전통. 추석은 한국의 문화와 역사를 중심으로 이루어진 공휴일입니다.
2024년 설날: 2월 10일 토요일; 2024년 설 연휴: 2월 9일 금요일 – 2월 12일 월요일